# Implementación Práctica de RAG con LangChain

## 1. Instalación de dependencias
```bash
!pip install langchain-community langchain-core langchain-ollama langchain-chroma langchain-google-genai pypdf python-dotenv
```

---

## 2. Cargar y procesar un documento PDF

In [1]:
from langchain_community.document_loaders import PyPDFLoader

def upload_pdf(path: str):
    try:
        loader = PyPDFLoader(path)
        pages = loader.lazy_load()
        text = ""
        for page in pages:
            text += page.page_content + "\n"
        return text
    except Exception as e:
        print(e)
        return ""
        
# Cambia 'documento.pdf' por el nombre de tu archivo PDF
raw_text = upload_pdf("documento.pdf")
print(raw_text[:500])  # Muestra los primeros 500 caracteres

Charles Perrault
BARBA AZUL
Charles Perrault (1628 - 1703) 
Imagen de dominio público. Fuente: 
http://es.wikipedia.org/wiki/Archivo:Barbebleue.jpg
Recursos de dominio público
Barba Azul
Charles Perrault
BARBA AZUL
Érase una vez un hombre que tenía hermosas casas en la ciudad y en el campo, vajilla de oro 
y plata, muebles forrados en finísimo brocado y carrozas todas doradas. Pero 
desgraciadamente, este hombre tenía la barba azul; esto le daba un aspecto tan feo y terrible 
que todas las mujer


## 3. Separación de texto (Text Splitter)

In [2]:
from langchain_text_splitters import CharacterTextSplitter

def text_splitter(text, chunk_size=2000, chunk_overlap=100, separator="\n"):
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separator=separator
    )
    docs = splitter.create_documents([text])
    return docs

docs = text_splitter(raw_text)
print(f"Cantidad de fragmentos: {len(docs)}")
print(docs[0].page_content[:300])

Cantidad de fragmentos: 6
Charles Perrault
BARBA AZUL
Charles Perrault (1628 - 1703) 
Imagen de dominio público. Fuente: 
http://es.wikipedia.org/wiki/Archivo:Barbebleue.jpg
Recursos de dominio público
Barba Azul
Charles Perrault
BARBA AZUL
Érase una vez un hombre que tenía hermosas casas en la ciudad y en el campo, vajilla 


## 4. Embeddings y Vector Store (Chroma)

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# Puedes cambiar el modelo de embedding aquí
embedding = OllamaEmbeddings(model="nomic-embed-text")

def get_vector_store(name_collection: str, docs):
    vector_store = Chroma(
        collection_name=name_collection,
        embedding_function=embedding,
        persist_directory="./vectorstore"
    )
    vector_store.add_documents(docs)
    return vector_store

vector_store = get_vector_store("langchain", docs)

## 5. Recuperación de documentos similares (Retrieval)

In [4]:
def retrieval(query: str, vector_store):
    docs = vector_store.similarity_search(query)
    return docs

# Ejemplo de pregunta
pregunta = "¿De qué trata el documento?"
resultados = retrieval(pregunta, vector_store)
for doc in resultados:
    print(doc.page_content[:300])

amaba desde hacía mucho tiempo; otra parte en comprar cargos de Capitán a sus dos 
hermanos; y el resto a casarse ella misma con un hombre muy correcto que la hizo olvidar los 
malos ratos pasados con Barba Azul.
MORALEJA
La curiosidad, teniendo sus encantos,
a menudo se paga con penas y con llantos
amaba desde hacía mucho tiempo; otra parte en comprar cargos de Capitán a sus dos 
hermanos; y el resto a casarse ella misma con un hombre muy correcto que la hizo olvidar los 
malos ratos pasados con Barba Azul.
MORALEJA
La curiosidad, teniendo sus encantos,
a menudo se paga con penas y con llantos
rato, pensando en la prohibición que le había hecho su marido, y temiendo que esta 
desobediencia pudiera acarrearle alguna desgracia. Pero la tentación era tan grande que no 
pudo superarla: tomó, pues, la llavecita y temblando abrió la puerta del gabinete.
Al principio no vio nada porque las venta
rato, pensando en la prohibición que le había hecho su marido, y temiendo que esta 
desobediencia

## 6. Prompt Template y LLM

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")

prompt = PromptTemplate.from_template("""
Eres un asistente experto en el contenido del documento proporcionado.
Utiliza siempre el contexto para responder.
Contexto: {contexto}
Pregunta del usuario: {input_user}
""")

def response(input_user: str, contexto: str, model_name="gemini-2.0-flash"):
    llm = ChatGoogleGenerativeAI(
        api_key=api_key,
        model=model_name,
        temperature=0.5
    )
    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        print(chunk.content, end="", flush=True)

# Uso:
contexto = "\n".join([doc.page_content for doc in resultados])
response(pregunta, contexto)

El documento trata sobre el cuento de Barba Azul. En el fragmento proporcionado, se describe cómo la esposa de Barba Azul desobedece su prohibición de entrar a un gabinete secreto. Al entrar, descubre los cadáveres de las anteriores esposas de Barba Azul. La llave del gabinete queda manchada de sangre y, a pesar de los intentos de la esposa por limpiarla, la mancha persiste debido a la naturaleza mágica de la llave. Barba Azul regresa antes de lo esperado y descubre la desobediencia de su esposa por la llave manchada de sangre. El documento también incluye dos moralejas que reflexionan sobre la curiosidad y sobre cómo los tiempos han cambiado, ya que ya no existen esposos tan terribles como Barba Azul.


## 7. Experimentación con modelos de embedding y LLM

In [6]:
embedding2 = OllamaEmbeddings(model="all-minilm")
vector_store2 = Chroma(
    collection_name="langchain2",
    embedding_function=embedding2,
    persist_directory="./vectorstore2"
)
vector_store2.add_documents(docs)
resultados2 = retrieval(pregunta, vector_store2)
contexto2 = "\n".join([doc.page_content for doc in resultados2])
response(pregunta, contexto2, model_name="gemini-2.5-flash")  # Cambia el modelo LLM aquí

El documento trata sobre fragmentos de una historia, aparentemente el cuento de Barba Azul. Describe la prohibición de un marido a su esposa de abrir un gabinete específico, la intensa curiosidad de ella por desobedecer esa orden y las riquezas de la casa.

También se menciona el desenlace para la esposa, quien olvida los malos ratos pasados con Barba Azul y se casa con un hombre muy correcto.

Finalmente, el documento presenta dos moralejas:
1.  Una sobre los peligros de la curiosidad, indicando que "a menudo se paga con penas y con llantos".
2.  Otra que reflexiona sobre el tiempo pasado de la historia, señalando que "ya no existe un esposo tan terrible" como el descrito.

## 8. Optimización del separador de texto

In [11]:
# Prueba con diferentes chunk_size, chunk_overlap y separadores
docs_alt = text_splitter(raw_text, chunk_size=1000, chunk_overlap=50, separator=r"\.|\n")
print(f"Fragmentos alternativos: {len(docs_alt)}")

Fragmentos alternativos: 1


## 9. Gestión avanzada de Chroma y otras bases vectoriales

In [ ]:
# Ejemplo de filtrado en Chroma (si tuvieras metadatos)
# vector_store.similarity_search(query, filter={"autor": "Nombre"})

# Investiga otras bases vectoriales compatibles:
# from langchain_faiss import FAISS
# from langchain_pinecone import Pinecone

## 10. Refinamiento de prompts

In [12]:
prompt2 = PromptTemplate.from_template("""
Eres un experto en el tema. Responde de forma concisa y cita la fuente si es posible.
Contexto: {contexto}
Pregunta: {input_user}
""")
# Puedes probar con prompt2 en la función response